##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Buscador de texto con Model Maker de TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_searcher"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/google/universal-sentence-encoder-lite/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo de TF Hub</a>
</td>
</table>

En este bloc de colab podrá aprender a usar la biblioteca [Model Maker de TensorFlow Lite](https://www.tensorflow.org/lite/models/modify/model_maker) para crear un modelo de Buscador TFLite. Puede usar un modelo de Buscador de texto para generar una Búsqueda Semática o una Respuesta Inteligente para su app. Este tipo de modelo le permite tomar una consulta de texto y buscar las entradas más relacionadas en un conjunto de datos de texto, como una base de datos de páginas web. El modelo devuelve una lista de las entradas con puntuaciones de menor distancia en el conjunto de datos, incluidos los metadatos que usted especifique, como la URL, el título de la página u otros identificadores de entrada de texto. Después de generarlo, puede implementarlo en dispositivos (por ejemplo, Android) utilizando [la API del Buscador de Bibliotecas de Tareas](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher) para ejecutar la inferencia con sólo unas pocas líneas de código.

Este tutorial utiliza el conjunto de datos de CNN/DailyMail como instancia para crear el modelo Buscador TFLite. Puede probar con su propio conjunto de datos con el formato de entrada compatible de valores separados por comas (CSV).

## Búsqueda de texto usando Vecino más cercano escalable

Este tutorial usa el conjunto de datos de resumen no anonimizado de CNN/DailyMail disponible públicamente, que se produjo a partir del [GitHub repo](https://github.com/abisee/cnn-dailymail). Este conjunto de datos contiene más de 300,000 artículos de noticias, siendo un buen conjunto de datos para construir el modelo Buscador, y devolver varias noticias relacionadas durante la inferencia del modelo para una consulta de texto.

El modelo Buscador de texto de este ejemplo usa un archivo de índice [ScaNN](https://github.com/google-research/google-research/tree/master/scann) (Scalable Nearest Neighbors) que puede buscar elementos similares a partir de una base de datos predefinida. ScaNN alcanza un rendimiento puntero para la búsqueda eficiente de similitudes vectoriales a escala.

Se usan Aspectos destacados y urls de este conjunto de datos en este colab para crear el modelo:

1. Aspectos destacados son el texto para generar los vectores de características de incorporación y luego usarlos para la búsqueda.
2. Las urls son el resultado devuelto que se muestra a los usuarios tras buscar los aspectos destacados relacionados.

Este tutorial guarda estos datos en el archivo CSV y, a continuación, usa el archivo CSV para construir el modelo. Aquí tiene varios ejemplos del conjunto de datos.

Aspectos destacados | Urls
--- | ---
Hawaiian Airlines aterriza de nuevo en el nº 1 en puntualidad. El informe sobre la calidad de las aerolíneas examina las 14 mayores compañías aéreas estadounidenses. ExpressJet <br> y American Airlines tuvieron el peor rendimiento en puntualidad. Virgin America tuvo la mejor gestión de equipajes; Southwest, el índice de denuncias más bajo. | http://www.cnn.com/2013/04/08/travel/airline-quality-report
El organismo rector del fútbol europeo revela la lista de países que pujan por albergar la fase final de 2020. La edición del 60º aniversario de la fase final será organizada por 13 <br> países. Treinta y dos países consideran la posibilidad de presentar sus pujas para albergar los partidos de 2020. La UEFA anunciará las sedes el 25 de septiembre. | http://edition.cnn.com:80/2013/09/20/sport/football/football-euro-2020-bid-countries/index.html?
El otrora cazador de pulpos Dylan Mayer también ha firmado ahora una petición de 5,000 buceadores para prohibir su caza en el parque Seacrest. La decisión del <br> Departamento de Pesca y Vida Silvestre podría tardar meses. | http://www.dailymail.co.uk:80/news/article-2238423/Dylan-Mayer-Washington-considers-ban-Octopus-hunting-diver-caught-ate-Puget-Sound.html?
Una galaxia fue observada 420 millones de años después del Big Bang, encontrada por el telescopio espacial Hubble de la NASA, el telescopio espacial Spitzer y uno de los <br> propios "lentes de aumento" de la naturaleza en el espacio. | http://www.dailymail.co.uk/sciencetech/article-2233883/The-furthest-object-seen-Record-breaking-image-shows-galaxy-13-3-BILLION-light-years-Earth.html


## Configuración


Comience instalando los paquetes necesarios, incluido el paquete Model Maker del repositorio [GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!pip install gdown

Importe los paquetes necesarios.

In [ ]:
from tflite_model_maker import searcher

### Preparar el conjunto de datos

Este tutorial usa el conjunto de datos de resumen de CNN / Daily Mail del repo [GitHub](https://github.com/abisee/cnn-dailymail).

Primero, descargue el texto y las urls de cnn y dailymail y descomprímalos. Si falla la descarga desde Google Drive, espere unos minutos para intentarlo de nuevo o descárguelo manualmente y luego súbalo al colab.

In [ ]:
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfTHk4NFg2SndKcjQ
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfM1BxdkxVaTY2bWs

!wget -O all_train.txt https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt
!tar xzf cnn_stories.tgz
!tar xzf dailymail_stories.tgz

Luego, guarde los datos en el archivo CSV que se puede cargar en la biblioteca `tflite_model_maker`. El código se basa en la lógica utilizada para cargar estos datos en [`tensorflow_datasets`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/summarization/cnn_dailymail.py). No podemos usar `tensorflow_dataset` directamente ya que no contiene urls que se usen en este colab.

Dado que se tarda mucho tiempo en procesar los datos para incorporar vectores de características para todo el conjunto de datos. Sólo el primer 5 % del conjunto de datos de la CNN y el Daily Mail se seleccionan de forma predeterminada para fines de demo. Puede ajustar la fracción o probar con el [model](https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/cnn_daily_text_searcher.tflite) TFLite precompilado con el 50 % de historias del conjunto de datos de CNN y Daily Mail para buscar también.

In [ ]:
#@title Save the highlights and urls to the CSV file
#@markdown Load the highlights from the stories of CNN / Daily Mail, map urls with highlights, and save them to the CSV file.

CNN_FRACTION = 0.05 #@param {type:"number"}
DAILYMAIL_FRACTION = 0.05 #@param {type:"number"}

import csv
import hashlib
import os
import tensorflow as tf

dm_single_close_quote = u"\u2019"  # unicode
dm_double_close_quote = u"\u201d"
END_TOKENS = [
    ".", "!", "?", "...", "'", "`", '"', dm_single_close_quote,
    dm_double_close_quote, ")"
]  # acceptable ways to end a sentence


def read_file(file_path):
  """Reads lines in the file."""
  lines = []
  with tf.io.gfile.GFile(file_path, "r") as f:
    for line in f:
      lines.append(line.strip())
  return lines


def url_hash(url):
  """Gets the hash value of the url."""
  h = hashlib.sha1()
  url = url.encode("utf-8")
  h.update(url)
  return h.hexdigest()


def get_url_hashes_dict(urls_path):
  """Gets hashes dict that maps the hash value to the original url in file."""
  urls = read_file(urls_path)
  return {url_hash(url): url[url.find("id_/") + 4:] for url in urls}


def find_files(folder, url_dict):
  """Finds files corresponding to the urls in the folder."""
  all_files = tf.io.gfile.listdir(folder)
  ret_files = []
  for file in all_files:
    # Gets the file name without extension.
    filename = os.path.splitext(os.path.basename(file))[0]
    if filename in url_dict:
      ret_files.append(os.path.join(folder, file))
  return ret_files


def fix_missing_period(line):
  """Adds a period to a line that is missing a period."""
  if "@highlight" in line:
    return line
  if not line:
    return line
  if line[-1] in END_TOKENS:
    return line
  return line + "."


def get_highlights(story_file):
  """Gets highlights from a story file path."""
  lines = read_file(story_file)

  # Put periods on the ends of lines that are missing them
  # (this is a problem in the dataset because many image captions don't end in
  # periods; consequently they end up in the body of the article as run-on
  # sentences)
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  highlight_list = []
  next_is_highlight = False
  for line in lines:
    if not line:
      continue  # empty line
    elif line.startswith("@highlight"):
      next_is_highlight = True
    elif next_is_highlight:
      highlight_list.append(line)

  # Make highlights into a single string.
  highlights = "\n".join(highlight_list)

  return highlights

url_hashes_dict = get_url_hashes_dict("all_train.txt")
cnn_files = find_files("cnn/stories", url_hashes_dict)
dailymail_files = find_files("dailymail/stories", url_hashes_dict)

# The size to be selected.
cnn_size = int(CNN_FRACTION * len(cnn_files))
dailymail_size = int(DAILYMAIL_FRACTION * len(dailymail_files))
print("CNN size: %d"%cnn_size)
print("Daily Mail size: %d"%dailymail_size)

with open("cnn_dailymail.csv", "w") as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=["highlights", "urls"])
  writer.writeheader()

  for file in cnn_files[:cnn_size] + dailymail_files[:dailymail_size]:
    highlights = get_highlights(file)
    # Gets the filename which is the hash value of the url.
    filename = os.path.splitext(os.path.basename(file))[0]
    url = url_hashes_dict[filename]
    writer.writerow({"highlights": highlights, "urls": url})


## Generar el modelo de Buscador de texto

Cree un modelo de Buscador de texto cargando un conjunto de datos, creando un modelo con los datos y exportando el modelo TFLite.

### Paso 1. Cargue el conjunto de datos

Model Maker toma el conjunto de datos de texto y los metadatos correspondientes de cada cadena de texto (como las urls en este ejemplo) en formato CSV. Incorpora las cadenas de texto en vectores de características usando el modelo de incorporación especificado por el usuario.

En esta demo, construimos el modelo Buscador utilizando [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder-lite/2), un modelo de incorporación de frases de última generación que ya está reentrenado desde [colab](https://github.com/tensorflow/tflite-support/blob/master/tensorflow_lite_support/examples/colab/on_device_text_to_image_search_tflite.ipynb). El modelo está optimizado para el rendimiento de la inferencia dentro del dispositivo, y sólo tarda 6 ms en incorporar una cadena de consulta (medido en el Pixel 6). Como alternativa, puede usar la versión cuantizada de [this](https://tfhub.dev/google/lite-model/universal-sentence-encoder-qa-ondevice/1?lite-format=tflite), que es más pequeña pero tarda 38 ms en cada incorporación.

In [ ]:
!wget -O universal_sentence_encoder.tflite https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/text_embedder.tflite

Cree una instancia de `searcher.TextDataLoader` y use el método `data_loader.load_from_csv` para cargar el conjunto de datos. Este paso tarda unos 10 minutos, ya que genera el vector de características de incorporación para cada texto uno por uno. También puede intentar cargar su propio archivo CSV y cargarlo para construir el modelo personalizado.

Especifique el nombre de la columna de texto y de la columna de metadatos en el archivo CSV.

- El texto se usa para generar los vectores de características de incorporación.
- Los metadatos son el contenido que se mostrará al buscar un texto determinado.

Aquí tiene las 4 primeras líneas del archivo CSV de CNN-DailyMail generado anteriormente.

Aspectos destacados | Urls
--- | ---
Funcionario sirio: Obama se encaramó a la copa del árbol y ahora no sabe cómo bajar. Obama envía una carta a los jefes de la Cámara de Representantes y del Senado. Obama <br> buscará la aprobación del Congreso para una acción militar contra Siria. El objetivo es determinar si se usaron armas químicas, no por quién, dice el portavoz de la ONU. | http://www.cnn.com/2013/08/31/world/meast/syria-civil-war/
Usain Bolt gana el tercer oro del campeonato mundial. Ancla a Jamaica en la victoria del relevo 4x100m. Octavo oro en los campeonatos para Bolt. Doblete de Jamaica en el relevo femenino de 4x100 metros. | http://edition.cnn.com/2013/08/18/sport/athletics-bolt-jamaica-gold
El empleado de la oficina de la agencia en Kansas City es uno de los cientos de trabajadores "virtuales". Los viajes de ida y vuelta del empleado a EE.UU. continental costaron el año pasado <br> más de 24,000 dólares. El programa de teletrabajo, como todas las prácticas de la GSA, está siendo revisado. | http://www.cnn.com:80/2012/08/23/politics/gsa-hawaii-teleworking
NUEVO: Una doctora canadiense dice que formó parte del equipo que examinó a Harry Burkhart en 2010. NUEVO: Diagnóstico: "autismo, ansiedad grave, trastorno <br> de estrés postraumático y depresión" Burkhart también es sospechoso en una investigación alemana sobre incendios provocados, según las autoridades. Los fiscales creen que el ciudadano alemán provocó una serie de incendios <br> en Los Angeles. | http://edition.cnn.com:80/2012/01/05/justice/california-arson/index.html?


In [ ]:
data_loader = searcher.TextDataLoader.create("universal_sentence_encoder.tflite", l2_normalize=True)
data_loader.load_from_csv("cnn_dailymail.csv", text_column="highlights", metadata_column="urls")

Para casos de uso de imágenes, puede crear una instancia `searcher.ImageDataLoader` y luego usar `data_loader.load_from_folder` para cargar imágenes desde la carpeta. La instancia `searcher.ImageDataLoader` debe ser creada por un modelo incorporador TFLite porque se aprovechará para codificar consultas en vectores de características y se exportará con el modelo Buscador TFLite. Por ejemplo:

```python
data_loader = searcher.ImageDataLoader.create("mobilenet_v2_035_96_embedder_with_metadata.tflite")
data_loader.load_from_folder("food/")
```

###Paso 2. Crear el modelo Buscador

- Configure las opciones de ScaNN. Consulte [la documentación de la api](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/searcher/ScaNNOptions) para más detalles.
- Cree el modelo Buscador a partir de los datos y las opciones ScaNN. Puede ver el [examen en profundidad](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) para saber más sobre el algoritmo ScaNN.

In [ ]:
scann_options = searcher.ScaNNOptions(
      distance_measure="dot_product",
      tree=searcher.Tree(num_leaves=140, num_leaves_to_search=4),
      score_ah=searcher.ScoreAH(dimensions_per_block=1, anisotropic_quantization_threshold=0.2))
model = searcher.Searcher.create_from_data(data_loader, scann_options)

En el ejemplo anterior, definimos las siguientes opciones:

- `distance_measure`: usamos el producto punto "dot_product" para medir la distancia entre dos vectores incorporados. Observe que en realidad calculamos el valor **negativo** del producto punto para preservar la noción de que "más pequeño es más cercano".

- `tree`: el conjunto de datos se divide el conjunto de datos en 140 particiones (aproximadamente la raíz cuadrada del tamaño de los datos), y se buscan 4 de ellas durante la recuperación, lo que equivale aproximadamente al 3 % del conjunto de datos.

- `score_ah`: cuantizamos los valores float incorporados a int8 con la misma dimensión para ahorrar espacio.

###Paso 3. Exporte el modelo TFLite

Entonces podrá exportar el modelo del Buscador TFLite.

In [ ]:
model.export(
      export_filename="searcher.tflite",
      userinfo="",
      export_format=searcher.ExportFormat.TFLITE)

## Pruebe el modelo TFLite en su consulta

Puede probar el modelo TFLite exportado usando un texto de consulta personalizado. Para consultar texto usando el modelo Buscador, inicialice el modelo y ejecute una búsqueda con frase de texto, como se indica a continuación:

In [ ]:
from tflite_support.task import text

# Initializes a TextSearcher object.
searcher = text.TextSearcher.create_from_file("searcher.tflite")

# Searches the input query.
results = searcher.search("The Airline Quality Rankings Report looks at the 14 largest U.S. airlines.")
print(results)

Consulte la [Documentación de la biblioteca de tareas](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher) para saber cómo integrar el modelo en varias plataformas.

# Más información

Para más información, consulte:

- [Guía](https://www.tensorflow.org/lite/models/modify/model_maker) y [Referencia de API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) de Model Maker de TensorFlow Lite.

- Biblioteca de tareas: [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher) para su implementación.

- Las apps de referencia de principio a fin: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_searcher/android).
